### Cross-Database Data Integration and Analysis Project - SPOTIFY 

In [1]:
#import necessary library packages:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import credentials
import os
import time 
import datetime
import requests
import numpy as np
import pandas as pd
import mysql.connector 
import pymysql
from sqlalchemy import create_engine
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from neo4j import GraphDatabase
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
from pyspark.sql.functions import col
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
#set up credentials to access Spotify APi data:
client_id = credentials.client_id
client_secret = credentials.client_secret


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

This code retrieves 10000 tracks of 2010 from Spotify. The fetched data is loaded into to databases: MySql database, MongoDB  and Neo4j graph database, also saved in CSV files.

#### Data fetching

In [3]:
#retrieving Spotify track data from 2010:
artist_id = []
artist = []
track_id = []
track = []
track_popularity = []
album = [] 
release_date = []

for i in range(0,1000,50):
    track_results = sp.search(q="year:2010", type="track", limit=50,offset=i)
    for i, t in enumerate(track_results["tracks"]["items"]):
        artist_id.append(t["artists"][0]["id"])
        artist.append(t["artists"][0]["name"])
        track_id.append(t["id"])
        track.append(t["name"])
        track_popularity.append(t["popularity"])
        album.append(t["album"]["name"])
        release_date.append(t["album"]["release_date"])

In [4]:
#creating pandas DataFrame:
data_df = pd.DataFrame({"artist_id":artist_id,"artist":artist,"track_id":track_id,"track":track,"track_popularity":track_popularity,"album":album,"release_date":release_date} )
print(data_df.shape)

(1000, 7)


In [5]:
data_df

,artist_id,artist,track_id,track,track_popularity,album,release_date
0,360IAlyVv4PCEVjgyMZrxK,Miguel,0JXXNGljqupsJaZsgSbMZV,Sure Thing,89,All I Want Is You,2010-11-26
1,4LLpKhyESsyAXpc4laK94U,Mac Miller,7FAFkQQZFeNwOFzTrSDFIh,The Spins,84,K.I.D.S. (Deluxe),2010-08-13
2,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,7BqBn9nzAq8spo5e7cZ0dJ,Just the Way You Are,86,Doo-Wops & Hooligans,2010-10-05
3,6f4XkbvYlXMH0QgVRzW0sM,Waka Flocka Flame,03tqyYWC9Um2ZqU0ZN849H,No Hands (feat. Roscoe Dash & Wale),79,Flockaveli,2010-10-01
4,5jTtGLk1mGFMY5lQOvJYUj,bôa,42qNWdLKCI41S4uzfamhFM,Duvet,84,Twilight,2010-04-20
...,...,...,...,...,...,...,...
995,2bGK38Qvf603fcckzUYYui,Whitey Morgan and the 78's,6LoM9k42hBelhv1kovisRf,Bad News,50,Whitey Morgan and the 78's,2010-10-12
996,5PokPZn11xzZXyXSfnvIM3,Lifehouse,5dePpbamNezj1qhHKJPViS,All In,49,Smoke & Mirrors (Deluxe),2010
997,4tZwfgrHOc3mvqYlEYSvVi,Daft Punk,0Jc2SfIHv63JNsUZpunh54,"Solar Sailer - From ""TRON: Legacy""/Score",56,TRON: Legacy,2010-12-03
998,3a9qv6NLHnsVxJUtKOMHvD,The Glitch Mob,5VWVVSABdgccboSdrKGiZH,Fortune Days,53,Drink the Sea,2010-06-08


In [6]:
#convert "release_date" to datetime format:
#data_df["release_date"] = pd.to_datetime(data_df["release_date"], format="%Y-%m-%d")

In [7]:
#add additional data to DataFrame:
artist_popularity = []
artist_genres = []
artist_followers = []

for a_id in data_df.artist_id:
    artist = sp.artist(a_id)
    artist_popularity.append(artist["popularity"])
    artist_genres.append(artist["genres"])
    artist_followers.append(artist["followers"]["total"])

In [18]:
data_df = data_df.assign(artist_popularity=artist_popularity, artist_genres=artist_genres, artist_followers=artist_followers)
data_df.head(5)

,artist_id,artist,track_id,track,track_popularity,album,release_date,artist_popularity,artist_genres,artist_followers
0,360IAlyVv4PCEVjgyMZrxK,Miguel,0JXXNGljqupsJaZsgSbMZV,Sure Thing,89,All I Want Is You,2010-11-26,77,[r&b],4786932
1,4LLpKhyESsyAXpc4laK94U,Mac Miller,7FAFkQQZFeNwOFzTrSDFIh,The Spins,84,K.I.D.S. (Deluxe),2010-08-13,82,"[hip hop, pittsburgh rap, pop rap, rap]",9493389
2,0du5cEVh5yTK9QJze8zA0C,Bruno Mars,7BqBn9nzAq8spo5e7cZ0dJ,Just the Way You Are,86,Doo-Wops & Hooligans,2010-10-05,87,"[dance pop, pop]",50912381
3,6f4XkbvYlXMH0QgVRzW0sM,Waka Flocka Flame,03tqyYWC9Um2ZqU0ZN849H,No Hands (feat. Roscoe Dash & Wale),79,Flockaveli,2010-10-01,64,"[atl hip hop, dirty south rap, pop rap, rap, s...",2714037
4,5jTtGLk1mGFMY5lQOvJYUj,bôa,42qNWdLKCI41S4uzfamhFM,Duvet,84,Twilight,2010-04-20,66,[],437622


In [116]:
#--->to be added as error occrued
#get track features:
acousticness = []
danceability = []
energy = []
instrumentalness = []
liveness = []
loudness = []
speechiness = []
tempo = []
time_signature = []

for features in data_df.track_id:
    track_features = sp.audio_features(track_id)
    acousticness.append(t["acousticness"])
    danceability.append["danceability"]
    energy.append(t[0]["energy"])
    instrumentalness.append(t[0]["instrumentalness"])
    liveness.append(t[0]["liveness"])
    loudness.append(t[0]["loudness"])
    speechiness.append(t[0]["speechiness"])
    tempo.append(t[0]["tempo"])
    time_signature.append(t[0]["time_signature"])

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=0JXXNGljqupsJaZsgSbMZV,7FAFkQQZFeNwOFzTrSDFIh,7BqBn9nzAq8spo5e7cZ0dJ,03tqyYWC9Um2ZqU0ZN849H,42qNWdLKCI41S4uzfamhFM,1eUGmzzvahJjOSWgDHuRlv,60IGhnH20N82dNxKnc8jDd,3hlksXnvbKogFdPbpO9vel,4HlFJV71xXKIGcU3kRyttv,2tJulUYLDKOg9XrtVkMgcJ,1UGD3lW3tDmgZfAVDh6w7r,0HPD5WQqrq7wPWR7P7Dw1i,161DnLWsx1i3u1JT05lzqU,3avYqdwHKEq8beXbeWCKqJ,0I1eFRytp4XRhLCjT6tZm7,2qYsSHsYkihWx043HVJQRV,2ENexcMEMsYk0rVJigVD3i,0V5cvmTKsYmF5FmGGEAfmS,3DK6m7It6Pw857FcQftMds,22L7bfCiAkJo5xGSQgmiIO,2a1o6ZejUi8U3wzzOtCOYw,0UV5zxRMz6AO4ZwUOZNIKI,10eBRyImhfqVvkiVEGf0N0,15JINEqzVMv3SvJTAXAKED,40gk32E7YaTFoQwDIWv2SY,5jzKL4BDMClWqRguW5qZvh,4356Typ82hUiFAynbLYbPn,7ySUcLPVX7KudhnmNcgY2D,2gZUPNdnz5Y45eiGxpHGSc,3GBApU0NuzH4hKZq4NOSdA,6epn3r7S14KUqlReYr77hA,22PMfvdz35fFKYnJyMn077,7LcfRTgAVTs5pQGEQgUEzN,6BdgtqiV3oXNqBikezwdvC,1gv4xPanImH17bKZ9rOveR,6KOEK6SeCEZOQkLj5M1PxH,2V4bv1fNWfTcyRJKmej6Sj,4AYX69oFP3UOS1CFmV9UfO,3fpTMuD1u3gJlVI4FadVHs,1fBl642IhJOE5U319Gy2Go,59dLtGBS26

In [10]:
#data_df = data_df.assign(acousticness=acousticness, danceability=danceability, energy=energy, instrumentalness=instrumentalness,liveness=liveness, loudness=loudness,speechiness=speechiness, tempo=tempo, time_signature=time_signature)
#data_df.head(20)

In [11]:
#creating pandas DataFrame for MySql database and saving to csv file:
track_df = data_df[["track","artist","album","track_popularity"]]
track_df.head(20)

,track,artist,album,track_popularity
0,Sure Thing,Miguel,All I Want Is You,89
1,The Spins,Mac Miller,K.I.D.S. (Deluxe),84
2,Just the Way You Are,Bruno Mars,Doo-Wops & Hooligans,86
3,No Hands (feat. Roscoe Dash & Wale),Waka Flocka Flame,Flockaveli,79
4,Duvet,bôa,Twilight,84
5,Girl With The Tattoo Enter.lewd,Miguel,All I Want Is You,82
6,Knee Deep (feat. Jimmy Buffett),Zac Brown Band,You Get What You Give,78
7,Super Bass,Nicki Minaj,Pink Friday (Complete Edition),82
8,"Hey, Soul Sister",Train,"Save Me, San Francisco (Golden Gate Edition)",85
9,Grenade,Bruno Mars,Doo-Wops & Hooligans,80


In [12]:
track_df.to_csv("spotify_mysql.csv")

In [13]:
#creating pandas DataFrame for MongoDB and saving to csv file::
artist_df = data_df[["artist","artist_genres","artist_popularity","artist_followers"]]
artist_df

,artist,artist_genres,artist_popularity,artist_followers
0,Miguel,[r&b],77,4786932
1,Mac Miller,"[hip hop, pittsburgh rap, pop rap, rap]",82,9493389
2,Bruno Mars,"[dance pop, pop]",87,50912381
3,Waka Flocka Flame,"[atl hip hop, dirty south rap, pop rap, rap, s...",64,2714037
4,bôa,[],66,437622
...,...,...,...,...
995,Whitey Morgan and the 78's,"[classic texas country, modern southern rock, ...",48,134439
996,Lifehouse,"[neo mellow, pop rock, post-grunge]",62,1971908
997,Daft Punk,"[electro, filter house, rock]",79,9641230
998,The Glitch Mob,"[glitch, glitch hop, indietronica]",59,555094


In [14]:
artist_df.to_csv("spotify_mongodb.csv")

In [15]:
#creating pandas DataFrame for Neo4j and saving to csv file:
artist_album_df = data_df[["artist","track","album"]]
artist_album_df

,artist,track,album
0,Miguel,Sure Thing,All I Want Is You
1,Mac Miller,The Spins,K.I.D.S. (Deluxe)
2,Bruno Mars,Just the Way You Are,Doo-Wops & Hooligans
3,Waka Flocka Flame,No Hands (feat. Roscoe Dash & Wale),Flockaveli
4,bôa,Duvet,Twilight
...,...,...,...
995,Whitey Morgan and the 78's,Bad News,Whitey Morgan and the 78's
996,Lifehouse,All In,Smoke & Mirrors (Deluxe)
997,Daft Punk,"Solar Sailer - From ""TRON: Legacy""/Score",TRON: Legacy
998,The Glitch Mob,Fortune Days,Drink the Sea


In [16]:
artist_album_df.to_csv("spotify_neo4j.csv")

#### Load data into databases:

In [19]:
#to Mysql:
connection = pymysql.connect(host="localhost", user="root",  password="", database="spotify_db")
cursor = connection.cursor()

create_table_query = """
    CREATE TABLE IF NOT EXISTS tracks (
        track_id INT AUTO_INCREMENT PRIMARY KEY,
        track VARCHAR(60),
        artist VARCHAR(60),
        album VARCHAR(60),
       track_popularity INT
        )
    """

cursor.execute(create_table_query)

In [ ]:
# Insert whole DataFrame into MySQL
track_df.to_sql("tracks", con = engine, if_exists = "append", chunksize = 1000, index=False)

In [ ]:
connection.commit()
connection.close()

In [22]:
#to MongoDB:
data = pd.read_csv("spotify_mongodb.csv")
uri = "uri_mongo"
client = MongoClient(uri, server_api=ServerApi('1'))

In [23]:
db = client["SpotifyDB"]
collection = db["Spotifydata"]

In [24]:
data.reset_index(inplace=True)
data_dict = data.to_dict("records")

In [26]:
collection.insert_many(data_dict)

In [ ]:
client.close()

In [ ]:
#to Neo4j:
uri = "neo4j+ssc://14c452fc.databases.neo4j.io"
username = "username"
password = "password"

In [ ]:
driver = GraphDatabase.driver(uri,auth=(username,password))

In [ ]:
def load_to_neo4j(session, data):
    with session.begin_transaction() as tx:
        for artist in data:
            create_artist_query = (
                """CREATE (artist:Artist{
                "name": $name,
                "track": $track,
                "album":$album})""")

    result = transaction.run(create_artist_query, {
                "name": artist["Artist"],
                "track": artist["Track"],
                "album": track["Album"]
})

In [ ]:
with driver.session() as session:
    print(result)

#### Extract data from databases:

In [ ]:
#MySQL:
import mysql.connector 

connection = mysql.connector.connect(host="localhost", user="root",  password="", database="spotify_db")

sql_select_query = "select * from tracks"
cursor = connection.cursor()
cursor.execute(sql_select_query)

records = cursor.fetchall()
records

In [ ]:
#MongoDB:
uri = "uri_mongo"
client = pymongo.MongoClient(uri, server_api=ServerApi('1'))
db = client["SpotifyDB"]
collection = db["Spotifydata"]

x = collection.find()
 
for data in x:
    print(data)

In [ ]:
#Neo4j: ---> not finished yet
#from py2neo import Graph,Node,Relationship

graph = Graph(<Database URL>, auth=(<Username>, <Password>))

def load_to_neo4j(session, data):
    with session.begin_transaction() as tx:
        for artist in data:
            create_artist_query = (
                """CREATE (artist:Artist{
                "name": $name,
                "track": $track,
                "album":$album})""")


tx.run(create_artist_query, {
                "name": artist["Artist"],
                "track": artist["Track"],
                "album": track["Album"]
})


#### Data transformation:

In [94]:
#clean and transform the extracted data using PySpark:
spark = SparkSession.builder.appName("Dataframe").getOrCreate()

In [95]:
path = ([
    r"C:\Users\krent\OneDrive\Desktop\PythonII\PythonIIPractical\FinalProject\spotify_mysql.csv",
    r"C:\Users\krent\OneDrive\Desktop\PythonII\PythonIIPractical\FinalProject\spotify_mongodb.csv",
    r"C:\Users\krent\OneDrive\Desktop\PythonII\PythonIIPractical\FinalProject\spotify_neo4j.csv"
])


In [96]:
data = spark.read.csv(path, header=True,inferSchema=True ) #--->does not read all csv files as expected at once.

In [97]:
data.show()

+---+-----------------+--------------------+-----------------+----------------+
|_c0|           artist|       artist_genres|artist_popularity|artist_followers|
+---+-----------------+--------------------+-----------------+----------------+
|  0|           Miguel|             ['r&b']|               77|         4786932|
|  1|       Mac Miller|['hip hop', 'pitt...|               82|         9493389|
|  2|       Bruno Mars|['dance pop', 'pop']|               87|        50912381|
|  3|Waka Flocka Flame|['atl hip hop', '...|               64|         2714037|
|  4|              bôa|                  []|               66|          437622|
|  5|           Miguel|             ['r&b']|               77|         4786932|
|  6|   Zac Brown Band|['contemporary co...|               72|         3695092|
|  7|      Nicki Minaj|['hip pop', 'pop'...|               86|        29510299|
|  8|            Train|['dance pop', 'ne...|               71|         4356359|
|  9|       Bruno Mars|['dance pop', 'po

In [98]:
#verify data types:
data.dtypes

[('_c0', 'int'),
 ('artist', 'string'),
 ('artist_genres', 'string'),
 ('artist_popularity', 'string'),
 ('artist_followers', 'string')]

In [99]:
#find count of Null, None, NaN of all data columns:
from pyspark.sql.functions import col,isnan, when, count
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+---+------+-------------+-----------------+----------------+
|_c0|artist|artist_genres|artist_popularity|artist_followers|
+---+------+-------------+-----------------+----------------+
|  0|     0|            0|                0|            1000|
+---+------+-------------+-----------------+----------------+



In [100]:
#replace column name:
renamed_df =data.withColumnRenamed("_c0","No").printSchema()

root
 |-- No: integer (nullable = true)
 |-- artist: string (nullable = true)
 |-- artist_genres: string (nullable = true)
 |-- artist_popularity: string (nullable = true)
 |-- artist_followers: string (nullable = true)



In [101]:
#check duplicate values in DataFrame:
data = data.distinct()
print("Distinct count: "+str(data.count()))
data.show(truncate=True)

Distinct count: 3000
+---+--------------------+--------------------+-----------------+----------------+
|_c0|              artist|       artist_genres|artist_popularity|artist_followers|
+---+--------------------+--------------------+-----------------+----------------+
| 21|             Dr. Dog|['chamber pop', '...|               61|          494432|
|189|         Clay Walker|['contemporary co...|               54|         1205559|
|409|        Alan Jackson|         ['country']|               71|         2480403|
|438|          Get Scared|                  []|               56|          642434|
|766|      Kendrick Lamar|['conscious hip h...|               87|        25336504|
|838|       Josh Thompson|    ['country road']|               42|          176875|
|854|              Common|['alternative hip...|               59|          904039|
|864|Two Door Cinema Club|['indie rock', 'i...|               67|         2341138|
|269|   Avenged Sevenfold|['alternative met...|               74| 

In [90]:
spark.stop()

#### 3. Data Integration:

In [105]:
#after processing data in PySpark convert to Pandas DataFrame:
pandas_df = data.toPandas()
pandas_df.head()

,_c0,artist,artist_genres,artist_popularity,artist_followers
0,21,Dr. Dog,"['chamber pop', 'indie rock', 'modern folk roc...",61,494432
1,189,Clay Walker,"['contemporary country', 'country', 'country r...",54,1205559
2,409,Alan Jackson,['country'],71,2480403
3,438,Get Scared,[],56,642434
4,766,Kendrick Lamar,"['conscious hip hop', 'hip hop', 'rap', 'west ...",87,25336504


In [108]:
#drop unecessary column:
data = pandas_df.drop(["_c0"], axis=1)

#### 4. Data Analysis:
   - Perform exploratory data analysis (EDA) to understand the characteristics of the integrated dataset (average values, standard deviation, etc.).
  

In [109]:
data

,artist,artist_genres,artist_popularity,artist_followers
0,Dr. Dog,"['chamber pop', 'indie rock', 'modern folk roc...",61,494432
1,Clay Walker,"['contemporary country', 'country', 'country r...",54,1205559
2,Alan Jackson,['country'],71,2480403
3,Get Scared,[],56,642434
4,Kendrick Lamar,"['conscious hip hop', 'hip hop', 'rap', 'west ...",87,25336504
...,...,...,...,...
2995,Rihanna,California King Bed,Loud (Japan Version),None
2996,Alfredo Olivas,Autorízame,El Patroncito,None
2997,Ozzy Osbourne,Let Me Hear You Scream,Scream (Expanded Edition),None
2998,Linkin Park,Waiting for the End,A Thousand Suns,None


In [113]:
#show data basic info:
data.info()
#data shape:
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist             3000 non-null   object
 1   artist_genres      3000 non-null   object
 2   artist_popularity  3000 non-null   object
 3   artist_followers   2000 non-null   object
dtypes: object(4)
memory usage: 93.9+ KB


(3000, 4)

In [114]:
#Describe the data
data.describe()

,artist,artist_genres,artist_popularity,artist_followers
count,3000,3000,3000,2000
unique,1384,1746,553,483
top,Taylor Swift,['pop'],64,56
freq,64,46,50,62


In [111]:
#Find the duplicates
data.duplicated().sum()

583

In [ ]:
#using Time Series, Linear regression or any other approach  to identify patterns, trends, and correlations.
#Correlation Matrix

#### 5. Graph Analysis:

#### 6. Data Visualization:

#### Top 10 most popular songs on Spotify 2010

In [115]:
sns.pairplot(data)
plt.show()

ValueError: No variables found for grid columns.

In [ ]:
top_10 = data_df.query("track_popularity > 90", inplace = False).sort_values("track_popularity").head(10)
top_10[:10]

#### Top 10 least popular songs on Spotify 20210

In [ ]:
least = data_df.sort_values("track_popularity", ascending = True).head(10)
least

In [ ]:
sns.pairplot(df)
plt.show()

#### 7. Insights and Reporting:

#### 8. Performance Optimization: